In [60]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

In [69]:
BASE_DIR = "/content/cat-and-dog"
train_dir = os.path.join(BASE_DIR, "training_set")
test_dir = os.path.join(BASE_DIR, "test_set")

IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [70]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    shuffle=False
)

Found 6404 images belonging to 2 classes.
Found 1601 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.


In [72]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

base_model = VGG16(
    weights="imagenet",
    include_top=False,
    input_shape=(128, 128, 3)
)

for layer in base_model.layers:
    layer.trainable = False


In [73]:
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")
])

In [74]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [75]:
history = model.fit(
    train_generator,
    epochs=3,
    validation_data=val_generator
)

Epoch 1/3
201/201 ━━━━━━━━━━━━━━━━━━━━ 1337s 7s/step - accuracy: 0.7835 - loss: 0.4726 - val_accuracy: 0.8651 - val_loss: 0.3010
Epoch 2/3
201/201 ━━━━━━━━━━━━━━━━━━━━ 1334s 7s/step - accuracy: 0.8608 - loss: 0.3101 - val_accuracy: 0.8576 - val_loss: 0.2996
Epoch 3/3
201/201 ━━━━━━━━━━━━━━━━━━━━ 1335s 7s/step - accuracy: 0.8987 - loss: 0.2481 - val_accuracy: 0.8938 - val_loss: 0.2451


In [76]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test Accuracy:", test_acc)

64/64 ━━━━━━━━━━━━━━━━━━━━ 337s 5s/step - accuracy: 0.8542 - loss: 0.3279
Test Accuracy: 0.8818586468696594


In [77]:
model.save("exercise_6_vgg16_CABALFIN.h5")